# Tutorial: Observables 

## Background

In this tutorial, we are going to load a pretrained model, use it to generate new samples, and calculate relevant observables based on these samples.

We consider a system of $N=L \times L$ atoms arranged on a square lattice.
The governing Hamiltonian defining the Rydberg atom array interactions has the following form:

$$
\hat{H} = \sum_{i<j} \frac{C_6}{\lVert \mathbf{r}_i - \mathbf{r}_j \rVert^6} \hat{n}_i \hat{n}_j -\delta \sum_{i=1}^N \hat{n}_i - \frac{\Omega}{2} \sum_{i=1}^N \hat{\sigma}^x_i. \quad (1)
$$

$$
    C_6 = \Omega \left( \frac{R_b}{a} \right)^6, \quad
    V_{ij} =  \frac{a^6}{\lVert \mathbf{r}_i - \mathbf{r}_j \rVert^6}, \quad (2)
$$

where $\hat{\sigma}^{x}_{i} = \vert g \rangle_i \langle r\vert_i + \vert r \rangle_i \langle g\vert_i $, the occupation number operator $\hat{n}_i = \frac{1}{2} \left( \hat{\sigma}_{i} + \mathbb{1} \right) =  \vert r\rangle_i \langle r \vert_i$ and $\hat{\sigma}_{i} = \vert r \rangle_i \langle r \vert_i - \vert g \rangle_i \langle g \vert_i$. The experimental settings of a Rydberg atom array are controlled by the detuning from resonance $\delta$, Rabi frequency $\Omega$, lattice length scale $a$ and the positions of the atoms $\{\mathbf{r}_i\}_i^N$. From equation (2) above, we obtain a symmetric matrix $\mathbf{V}$, that encapsulates the relevant information about the lattice geometry, and derive the Rydberg blockade radius $R_b$, within which simultaneous excitations are penalized. Finally, for the purposes of our study, the atom array is considered to be affected by thermal noise, in equilibrium at a temperature $T$. The experimental settings are thus captured by the set of parameters $\mathbf{x} = (\Omega, \delta/\Omega, R_b/a, \mathbf{V}, \beta / \Omega)$, where $\beta$ is the inverse temperature. 

In [23]:
import os

import torch
from rydberggpt.models.rydberg_encoder_decoder import get_rydberg_graph_encoder_decoder
from rydberggpt.models.utils import generate_prompt
from rydberggpt.observables.rydberg_energy import (
    get_rydberg_energy,
    get_staggered_magnetization,
    get_x_magnetization,
)
from rydberggpt.utils import create_config_from_yaml, load_yaml_file
from rydberggpt.utils_ckpt import get_model_from_ckpt
from torch_geometric.data import Batch

## Loading the model

We have three pretrained models trained on three different datasets. Model $M_1$ is trained with data for systems of size $L=5,6$ (`models/ds_1`); Model $M_2$ utilizes datasets for systems with $L=5,6,11,12$ (`models/ds_2`); and Model $M_3$ is trained on data covering  $L=5,6,11,12,15,16$ (`models/ds_3`).

Let's start by loading the pretrained model and setting it into eval mode to ensure that the dropout layers are disabled.

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"

base_path = os.path.abspath("../")
log_path = os.path.join(base_path, "models/ds_1/")

yaml_dict = load_yaml_file(log_path, "hparams.yaml")
config = create_config_from_yaml(yaml_dict)

model = get_model_from_ckpt(
    log_path, model=get_rydberg_graph_encoder_decoder(config), ckpt="best"
)
model.to(device=device)
model.eval()  # don't forget to set to eval mode

RydbergEncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=32, out_features=128, bias=True)
          (w_2): Linear(in_features=128, out_features=32, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0-1): 2 x SublayerConnection(
            (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Decoder(
    (layers): ModuleList(
      (0-2): 3 x DecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_fea

## Generating system prompt and samples

Next, let us define our system prompt $\mathbf{x} = (\Omega, \delta/\Omega, R_b/a, \mathbf{V}, \beta / \Omega)$. Below is a function `generate_prompt` that generates the required prompt structure to query the trained model. The prompt is a graph structure capturing the relevant information about the system, such as the lattice geometry, the Rydberg blockade radius, the temperature, and the Rabi frequency. The function `generate_samples` generates samples from the model given the prompt.

### System prompt

In [25]:
L = 5
delta = 1.0
omega = 1.0
beta = 64.0
Rb = 1.15
num_samples = 5

pyg_graph = generate_prompt(
    model_config=config,
    n_rows=L,
    n_cols=L,
    delta=delta,
    omega=omega,
    beta=beta,
    Rb=Rb,
)

### Generating samples

The sampling function requires a batch of prompts; therefore, we duplicate our `pyg_graph` prompts as many times as we want to generate samples. The reasoning behind this is to allow the model to generate samples in parallel for different Hamiltonian parameters. This is especially helpful when training variationally.

In [26]:
# duplicate the prompt for num_samples
cond = [pyg_graph for _ in range(num_samples)]
cond = Batch.from_data_list(cond)

samples = model.get_samples(
    batch_size=len(cond), cond=cond, num_atoms=L**2, fmt_onehot=False
)

Generating atom 25/25                                                          


## Observables 

Now we are ready to calculate observables based on the samples generated. We consider three observables: the stagger-magnetization, x-magnetization and the Rydberg energy.
### Rydberg energy 

We consider an estimate of the ground state energy $\langle E \rangle$, which is defined as

$$
\langle E \rangle  
\approx \frac{1}{N_s} \sum_{\boldsymbol{\sigma} \sim p_{\theta}(\boldsymbol{\sigma};\mathbf{x})} \frac{\langle \boldsymbol{\sigma}|\widehat{H}|\Psi_{\theta}\rangle}{\langle \boldsymbol{\sigma}|\Psi_{\theta}\rangle}.
$$

We provide a function `get_rydberg_energy` that calculates the Rydberg energy of the samples generated. Note that this fn requires a single prompt.

In [27]:
energy = get_rydberg_energy(model, samples, cond=pyg_graph, device=device)
print(energy.mean() / L**2)

tensor(0.0244)


### Stagger magnetization

The staggered magnetization for the square-lattice Rydberg array defined in its occupation basis.  This quantity is the order parameter for the disorder-to-checkerboard quantum phase transition, and can be calculated simply with 

$$
    \langle\hat{\sigma}^{\text{stag}}\rangle  \approx \frac{1}{N_s} \sum_{\boldsymbol{\sigma} \sim p_\theta(\boldsymbol{\sigma};\mathbf{x})}
    \left|   \sum_{i=1}^{N} (-1)^i  \frac{n_i(\boldsymbol{\sigma}) - 1/2}{N} \right| ,
$$

where $i$ runs over all $N = L \times L$ atoms and $n_i(\boldsymbol{\sigma}) = \langle \boldsymbol{\sigma}| r_i \rangle\langle r_i|\boldsymbol{\sigma} \rangle$ is the occupation number operator acting on atom $i$ in a given configuration $\boldsymbol{\sigma}$. 
Because this observable is diagonal, it can be computed directly from samples inferred from the decoder. The outer sum shows how importance sampling is used to estimate the expectation value over this operator, approximating the probability of a given configuration with the frequency with which it is sampled. 


We provide a function `get_staggered_magnetization` that calculates the stagger magnetization of the samples generated.

In [28]:
staggered_magnetization = get_staggered_magnetization(samples, L, L, device=device)
print(staggered_magnetization.mean() / L**2)

tensor(0.0144)


### X-magnetization

We consider an off-diagonal observable, where we must make use of the ground state wave function amplitudes of the inferred samples $\Psi(\boldsymbol{\sigma}) = \sqrt{p_{\theta}(\boldsymbol{\sigma})}$. 
As an example, we examine the spatially averaged expectation value of $\hat{\sigma}_x$, which is defined as

$$
    \langle \hat{\sigma}^x \rangle  \approx \frac{1}{N_s} \sum_{\boldsymbol{\sigma} \sim p_\theta(\boldsymbol{\sigma};\mathbf{x})} \frac{1}{N}
    \sum_{\boldsymbol{\sigma}' \in \mathrm{SSF}(\boldsymbol{\sigma})} \frac{\Psi_\theta(\boldsymbol{\sigma}')}{\Psi_\theta(\boldsymbol{\sigma})},
$$

where the variable $\left\{\boldsymbol{\sigma'}\right\}$ is the set of configurations that are connected to $\boldsymbol{\sigma}$ by a single spin flip (SSF).

We provide a function `get_x_magnetization` that calculates the stagger magnetization of the samples generated.
Note that we do not have to batch our prompt. The energy is calculated for a single system prompt.

In [29]:
x_magnetization = get_x_magnetization(model, samples, cond=pyg_graph, device=device)
print(x_magnetization.mean() / L**2)

tensor(0.7578)
